In [14]:
# %matplotlib widget

from __future__ import annotations

import re
from collections import defaultdict
from dataclasses import dataclass, field
from itertools import permutations, product
from math import inf
from random import choice

import matplotlib.colors as mcolors
import matplotlib.pyplot as plt
import networkx as nx
import numpy as np
import numpy.typing as npt
from mpl_toolkits.mplot3d import axes3d
from numpy import int_, object_
from numpy.typing import NDArray
from test_utilities import test
from util import *

COLORS = list(mcolors.CSS4_COLORS.keys())

<link href="style.css" rel="stylesheet"></link>
<article class="day-desc"><h2>--- Day 23: Unstable Diffusion ---</h2><p>You enter a large crater of gray dirt where the grove is supposed to be. All around you, plants you imagine were expected to be full of fruit are instead withered and broken. A large group of Elves has formed in the middle of the grove.</p>
<p>"...but this volcano has been dormant for months. Without ash, the fruit can't grow!"</p>
<p>You look up to see a massive, snow-capped mountain towering above you.</p>
<p>"It's not like there are other active volcanoes here; we've looked everywhere."</p>
<p>"But our scanners show active magma flows; clearly it's going <em>somewhere</em>."</p>
<p>They finally notice you at the edge of the grove, your pack almost overflowing from the random <em class="star">star</em> fruit you've been collecting. Behind you, elephants and monkeys explore the grove, looking concerned. Then, the Elves recognize the ash cloud slowly spreading above your recent detour.</p>
<p>"Why do you--" "How is--" "Did you just--"</p>
<p>Before any of them can form a complete question, another Elf speaks up: "Okay, new plan. We have almost enough fruit already, and ash from the plume should spread here eventually. If we quickly plant new seedlings now, we can still make it to the extraction point. Spread out!"</p>
<p>The Elves each reach into their pack and pull out a tiny plant. The plants rely on important nutrients from the ash, so they can't be planted too close together.</p>
<p>There isn't enough time to let the Elves figure out where to plant the seedlings themselves; you quickly scan the grove (your puzzle input) and note their positions.</p>
<p>For example:</p>
<pre><code>....#..
..###.#
#...#.#
.#...##
#.###..
##.#.##
.#..#..
</code></pre>
<p>The scan shows Elves <code>#</code> and empty ground <code>.</code>; outside your scan, more empty ground extends a long way in every direction. The scan is oriented so that <em>north is up</em>; orthogonal directions are written N (north), S (south), W (west), and E (east), while diagonal directions are written NE, NW, SE, SW.</p>
<p>The Elves follow a time-consuming process to figure out where they should each go; you can speed up this process considerably. The process consists of some number of <em>rounds</em> during which Elves alternate between considering where to move and actually moving.</p>
<p>During the <em>first half</em> of each round, each Elf considers the eight positions adjacent to themself. If no other Elves are in one of those eight positions, the Elf <em>does not do anything</em> during this round. Otherwise, the Elf looks in each of four directions in the following order and <em>proposes</em> moving one step in the <em>first valid direction</em>:</p>
<ul>
<li>If there is no Elf in the N, NE, or NW adjacent positions, the Elf proposes moving <em>north</em> one step.</li>
<li>If there is no Elf in the S, SE, or SW adjacent positions, the Elf proposes moving <em>south</em> one step.</li>
<li>If there is no Elf in the W, NW, or SW adjacent positions, the Elf proposes moving <em>west</em> one step.</li>
<li>If there is no Elf in the E, NE, or SE adjacent positions, the Elf proposes moving <em>east</em> one step.</li>
</ul>
<p>After each Elf has had a chance to propose a move, the <em>second half</em> of the round can begin. Simultaneously, each Elf moves to their proposed destination tile if they were the <em>only</em> Elf to propose moving to that position. If two or more Elves propose moving to the same position, <em>none</em> of those Elves move.</p>
<p>Finally, at the end of the round, the <em>first direction</em> the Elves considered is moved to the end of the list of directions. For example, during the second round, the Elves would try proposing a move to the south first, then west, then east, then north. On the third round, the Elves would first consider west, then east, then north, then south.</p>
<p>As a smaller example, consider just these five Elves:</p>
<pre><code>.....
..##.
..#..
.....
..##.
.....
</code></pre>
<p>The northernmost two Elves and southernmost two Elves all propose moving north, while the middle Elf cannot move north and proposes moving south. The middle Elf proposes the same destination as the southwest Elf, so neither of them move, but the other three do:</p>
<pre><code>..##.
.....
..#..
...#.
..#..
.....
</code></pre>
<p>Next, the northernmost two Elves and the southernmost Elf all propose moving south. Of the remaining middle two Elves, the west one cannot move south and proposes moving west, while the east one cannot move south <em>or</em> west and proposes moving east. All five Elves succeed in moving to their proposed positions:</p>
<pre><code>.....
..##.
.#...
....#
.....
..#..
</code></pre>
<p>Finally, the southernmost two Elves choose not to move at all. Of the remaining three Elves, the west one proposes moving west, the east one proposes moving east, and the middle one proposes moving north; all three succeed in moving:</p>
<pre><code>..#..
....#
#....
....#
.....
..#..
</code></pre>
<p>At this point, no Elves need to move, and so the process ends.</p>
<p>The larger example above proceeds as follows:</p>
<pre><code>== Initial State ==
..............
..............
.......#......
.....###.#....
...#...#.#....
....#...##....
...#.###......
...##.#.##....
....#..#......
..............
..............
..............

== End of Round 1 ==
..............
.......#......
.....#...#....
...#..#.#.....
.......#..#...
....#.#.##....
..#..#.#......
..#.#.#.##....
..............
....#..#......
..............
..............

== End of Round 2 ==
..............
.......#......
....#.....#...
...#..#.#.....
.......#...#..
...#..#.#.....
.#...#.#.#....
..............
..#.#.#.##....
....#..#......
..............
..............

== End of Round 3 ==
..............
.......#......
.....#....#...
..#..#...#....
.......#...#..
...#..#.#.....
.#..#.....#...
.......##.....
..##.#....#...
...#..........
.......#......
..............

== End of Round 4 ==
..............
.......#......
......#....#..
..#...##......
...#.....#.#..
.........#....
.#...###..#...
..#......#....
....##....#...
....#.........
.......#......
..............

== End of Round 5 ==
.......#......
..............
..#..#.....#..
.........#....
......##...#..
.#.#.####.....
...........#..
....##..#.....
..#...........
..........#...
....#..#......
..............
</code></pre>

<p>After a few more rounds...</p>
<pre><code>== End of Round 10 ==
.......#......
...........#..
..#.#..#......
......#.......
...#.....#..#.
.#......##....
.....##.......
..#........#..
....#.#..#....
..............
....#..#..#...
..............
</code></pre>
<p>To make sure they're on the right track, the Elves like to check after round 10 that they're making good progress toward covering enough ground. To do this, count the number of empty ground tiles contained by the smallest rectangle that contains every Elf. (The edges of the rectangle should be aligned to the N/S/E/W directions; the Elves do not have the patience to calculate <span title="Arbitrary Rectangles is my Piet Mondrian cover band.">arbitrary rectangles</span>.) In the above example, that rectangle is:</p>
<pre><code>......#.....
..........#.
.#.#..#.....
.....#......
..#.....#..#
#......##...
....##......
.#........#.
...#.#..#...
............
...#..#..#..
</code></pre>
<p>In this region, the number of empty ground tiles is <code><em>110</em></code>.</p>
<p>Simulate the Elves' process and find the smallest rectangle that contains the Elves after 10 rounds. <em>How many empty ground tiles does that rectangle contain?</em></p>
</article>


In [15]:
from collections import Counter

from more_itertools import minmax


tests = [
    {
        "name": "Smaller Example",
        "s": """
        .....
        ..##.
        ..#..
        .....
        ..##.
        .....
       """,
        "rounds": 3,
        "expected": 25,
    },
    {
        "name": "Example",
        "s": """
            ....#..
            ..###.#
            #...#.#
            .#...##
            #.###..
            ##.#.##
            .#..#..
        """,
        "rounds": 10,
        "expected": 110,
    },
]

type Elf = tuple[int, int]


class Scan:
    proposals = [
        # If there is no Elf in the N, NE, or NW adjacent positions, the Elf proposes moving north one step.
        (((-1, 0), (-1, -1), (-1, 1)), (-1, 0)),
        # If there is no Elf in the S, SE, or SW adjacent positions, the Elf proposes moving south one step.
        (((1, 0), (1, -1), (1, 1)), (1, 0)),
        # If there is no Elf in the W, NW, or SW adjacent positions, the Elf proposes moving west one step.
        (((0, -1), (-1, -1), (1, -1)), (0, -1)),
        # If there is no Elf in the E, NE, or SE adjacent positions, the Elf proposes moving east one step
        (((0, 1), (-1, 1), (1, 1)), (0, 1)),
    ]

    around = (-1, -1), (-1, 0), (-1, 1), (0, 1), (1, 1), (1, 0), (1, -1), (0, -1)

    def __init__(self, s: str) -> None:
        self.elfs = {
            (r, m.start())
            for r, l in enumerate(s.strip().splitlines())
            for m in re.finditer(r"#", l.strip())
        }

    def rounds(self, rounds: int, debug: bool = False) -> None:
        if debug:
            print(f"== Initial State ==")
            print(self)

        for round in range(rounds):
            self.elfs = self._round(round)

            if debug:
                print()
                print(f"== End of round {round+1} ==")
                print(self)

    def _round(self, round: int) -> set[Elf]:
        porposed = self._proposed(round)
        return self._move(porposed)

    def _move(self, proposed: list[tuple[Elf, Elf]]) -> set[Elf]:
        doubles = {k for k, v in Counter(to for _, to in proposed).items() if v > 1}
        return {fr if to in doubles or to is None else to for fr, to in proposed}

    def _proposed(self, round: int) -> list[tuple[Elf, Elf | None]]:
        n = len(self.proposals)
        round %= n

        proposed = []

        for r, c in self.elfs:
            if all((r + ddr, c + ddc) not in self.elfs for ddr, ddc in self.around):
                proposed.append(((r, c), None))
            else:
                touched = False
                for d in range(n):
                    idx = (round + d) % n
                    conditions, (dr, dc) = self.proposals[idx]
                    if all(
                        (r + ddr, c + ddc) not in self.elfs for ddr, ddc in conditions
                    ):
                        touched = True
                        proposed.append(((r, c), (r + dr, c + dc)))
                        break

                if not touched:
                    proposed.append(((r, c), None))

        return proposed

    def _empty_ground_tiles(self) -> int:
        min_row, max_row = minmax(r for r, _ in self.elfs)
        min_col, max_col = minmax(c for _, c in self.elfs)
        return sum(
            1
            for r, c in product(
                range(min_row, max_row + 1), range(min_col, max_col + 1)
            )
            if (r, c) not in self.elfs
        )

    def __str__(self) -> str:
        min_row, max_row = minmax(r for r, _ in self.elfs)
        min_col, max_col = minmax(c for _, c in self.elfs)
        border = 3

        return "\n".join(
            "".join(
                "#" if (r, c) in self.elfs else "."
                for c in range(min_col - border, max_col + 1 + border)
            )
            for r in range(min_row - border, max_row + 1 + border)
        )

    @classmethod
    def empty_ground_tiles(cls, s: str, rounds: int, debug=False) -> int:
        scan = Scan(s)
        scan.rounds(rounds, debug)
        return scan._empty_ground_tiles()


@test(tests=tests)
def test_part_I(s: str, rounds: int) -> int:
    return Scan.empty_ground_tiles(s, rounds)


Test Smaller Example passed, for test_part_I.
Test Example passed, for test_part_I.
Success


In [16]:
with open("../input/day23.txt") as facing:
    puzzle = facing.read()

print(f"Part I: {Scan.empty_ground_tiles(puzzle, 10)}")

Part I: 3906


<link href="style.css" rel="stylesheet"></link>
<main>

<p>Your puzzle answer was <code>3906</code>.</p><p class="day-success">The first half of this puzzle is complete! It provides one gold star: *</p>


<link href="style.css" rel="stylesheet"></link>
<article class="day-desc"><h2 id="part2">--- Part Two ---</h2><p>It seems you're on the right track. Finish simulating the process and figure out where the Elves need to go. How many rounds did you save them?</p>
<p>In the example above, the <em>first round where no Elf moved</em> was round <code><em>20</em></code>:</p>
<pre><code>.......#......
....#......#..
..#.....#.....
......#.......
...#....#.#..#
#.............
....#.....#...
..#.....#.....
....#.#....#..
.........#....
....#......#..
.......#......
</code></pre>
<p>Figure out where the Elves need to go. <em>What is the number of the first round where no Elf moves?</em></p>
</article>


In [17]:
tests = [
    {
        "name": "Example",
        "s": """
            ....#..
            ..###.#
            #...#.#
            .#...##
            #.###..
            ##.#.##
            .#..#..
        """,
        "expected": 20,
    },
]


class ScanII(Scan):
    def _first_round_no_elfs_moved(self) -> int:
        round = 0
        while True:
            elfs = self._round(round)

            if self.elfs == elfs:
                return round + 1

            self.elfs = elfs
            round += 1

    @classmethod
    def first_round_no_elfs_moved(cls, s: str) -> int:
        scan = cls(s)
        return scan._first_round_no_elfs_moved()


@test(tests=tests)
def test_part_II(s: str) -> int:
    return ScanII.first_round_no_elfs_moved(s)


Test Example passed, for test_part_II.
Success


In [18]:
print(f"PartI II: {ScanII.first_round_no_elfs_moved(puzzle)}")

PartI II: 895


<link href="style.css" rel="stylesheet"></link>

<main>

<p>Your puzzle answer was <code>895</code>.</p><p class="day-success">Both parts of this puzzle are complete! They provide two gold stars: **</p>

</main>
